In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
# %watermark -p numpy,sklearn,pandas
# %watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
# %watermark -p torch,torchvision,torchaudio
# %watermark -p tensorflow,tensorboard,tflite
# %watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
# %matplotlib inline
# %config InlineBackend.figure_format='retina'
# %config IPCompleter.use_jedi = False

# from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
# from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import argparse, shlex, signal, traceback
import numpy as np

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x, debug=False):
    try:
        x = x.strip()
        if x[0] == '/' or x[1] == '/':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                x = 'https://' + x
                x = requests.get(x)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                x = x.replace('github.com', 'raw.githubusercontent.com')
                mod = x.split('/')
                for s in ['/main/', '/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(x)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    if debug:
                        print(x)
                    x = requests.get(x)
                    if x.status_code == 200:
                        x = x.text
                        break
        if debug:
            return x
        else:
            exec(x, globals())
    except Exception as err:
        # sys.stderr.write(f'request {x} : {err}')
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)
    

###
### Display ###
###

_IMPORT('import pandas as pd')
_IMPORT('import cv2')
_IMPORT('from PIL import Image')
_IMPORT('import matplotlib.pyplot as plt')
_IMPORT('import plotly')
_IMPORT('import plotly.graph_objects as go')
_IMPORT('import ipywidgets as widgets')
_IMPORT('from ipywidgets import interact, interactive, fixed, interact_manual')

# plotly.offline.init_notebook_mode(connected=False)

plt.rcParams['figure.figsize'] = (12.0, 8.0)

def show_image(imgsrc, width=None, height=None):
    if isinstance(imgsrc, np.ndarray):
        img = imgsrc
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if imgsrc.startswith('http'):
        data_url = imgsrc
    else:
        if len(imgsrc) > 2048:
            data_url = 'data:image/jpg;base64,' + imgsrc
        else:
            img = open(imgsrc, 'rb').read()
            data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<center><img %s %s src="%s"/></center>' % (W, H, data_url))


class COLORS(object):
    # BGR
    BLUE       = (255 , 0   , 0)
    GREEN      = (0   , 255 , 0)
    RED        = (0   , 0   , 255)
    BLACK      = (0   , 0   , 0)
    YELLOW     = (0   , 255 , 255)
    WHITE      = (255 , 255 , 255)
    CYAN       = (255 , 255 , 0)
    MAGENTA    = (255 , 0   , 242)
    GOLDEN     = (32  , 218 , 165)
    LIGHT_BLUE = (255 , 9   , 2)
    PURPLE     = (128 , 0   , 128)
    CHOCOLATE  = (30  , 105 , 210)
    PINK       = (147 , 20  , 255)
    ORANGE     = (0   , 69  , 255)

In [2]:
import json, functools, datetime

class __JsonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (datetime.datetime, datetime.timedelta)):
            return '{}'.format(obj)
        elif isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return json.JSONEncoder.default(self, obj)

json.dumps = functools.partial(json.dumps, cls=__JsonEncoder)

In [3]:
import skimage.filters as filters
import threading
import shutil
import glob
import pickle
import collections
from enum import IntEnum
# _IMPORT('gitee.com/qrsforever/nb_easy/easy_widget')
_IMPORT('./easy_widget.py')

In [4]:
g_ctx = None

## Generate Grid Aurco

In [5]:
# A4_width_mm, A4_height_mm, printer_dpi = 210, 297, 600
printer_dpi = 600
printer_dpmm = printer_dpi / 25.4
marker_size_mm, gap_size_mm = 25, 3
aurco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)

landscape = True # portrait

if landscape:
    A4_width_mm, A4_height_mm, printer_dpi = 280, 210, 600
else:
    A4_width_mm, A4_height_mm, printer_dpi = 210, 280, 600
    
markersX = (A4_width_mm - gap_size_mm) // (marker_size_mm + gap_size_mm)
markersY = (A4_height_mm - gap_size_mm) // (marker_size_mm + gap_size_mm)

aruco_gridboard = cv2.aruco.GridBoard_create(markersX, markersY, marker_size_mm, gap_size_mm, aurco_dict)

width = markersX * marker_size_mm  + (markersX - 1) * gap_size_mm
height =  markersY * marker_size_mm + (markersY - 1) * gap_size_mm

cv2.imwrite("aruco_gridboard.png", aruco_gridboard.draw((int(printer_dpmm * width), int(printer_dpmm * height))))
print(markersX, markersY)

9 7


## Camera Calibration

In [29]:
class CalibrateState(IntEnum):
    COLLECT = 1
    READIMAGE = 2
    CALIBRATE = 3
    COMPLETED = 4

def stop_camera_calibrate(ctx, w_btn_source=None):
    ctx.logger('stop_camera_calibrate()')
    if hasattr(ctx, 'camera') and ctx.camera:
        ctx.camera_is_running = False
        for i in range(3):
            if ctx.camera is None:
                break
            time.sleep(1)
        if ctx.camera:
            ctx.camera.release()

def save_calibration_intrinsics(camera_matrix, dist_coeffs, out_dir):
    cv_file = cv2.FileStorage(f'{out_dir}/intrinsics.yaml', cv2.FILE_STORAGE_WRITE)
    cv_file.write('camera_matrix', camera_matrix)
    cv_file.write('dist_coeffs', dist_coeffs)
    cv_file.release()

def load_calibration_intrinsics(out_dir):
    cv_file = cv2.FileStorage(f'{out_dir}/intrinsics.yaml', cv2.FILE_STORAGE_READ)
    camera_matrix = cv_file.getNode('camera_matrix').mat()
    dist_coeffs = cv_file.getNode('dist_coeffs').mat()
    return camera_matrix, dist_coeffs
 
def start_calibrate_camera_with_good_flags(ctx, calibrate, img_size):
    flags = 0
    K = np.array([[max(img_size), 0, img_size[1]/2],[0, max(img_size), img_size[0]/2], [0, 0, 1]])
    D = np.zeros((5, 1))
    
    retval, mat, dist, rvecs, tvecs, \
    std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)

    ctx.logger(json.dumps({
        're-project-error': retval, 
        'mat': str(mat.round(3).tolist()),
        'dist': str(dist.round(3).tolist())}, indent=4))

    aspect_ratio = mat[0][0] / mat[1][1]
    if 1.0 - min(aspect_ratio, 1.0/aspect_ratio) < 0.01:
        flags += cv2.CALIB_FIX_ASPECT_RATIO
        retval, mat, dist, rvecs, tvecs, \
        std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)
        ctx.logger(json.dumps({
            'aspect_ratio': aspect_ratio,
            're-project-error': retval, 
            'mat': str(mat.round(3).tolist()),
            'dist': str(dist.round(3).tolist())}, indent=4))

    center_point_reldiff = max(abs(np.array(mat[0, 2], mat[1][2]) - np.array(img_size)/2) / np.array(img_size))
    if center_point_reldiff < 0.05:
        flags += cv2.CALIB_FIX_PRINCIPAL_POINT
        retval, mat, dist, rvecs, tvecs, \
        std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)
        ctx.logger(json.dumps({
            'center_point_reldiff': center_point_reldiff,
            're-project-error': retval, 
            'mat': str(mat.round(3).tolist()),
            'dist': str(dist.round(3).tolist())}, indent=4))

    error_threshold = 1.25 * retval
    camera_matrix = mat
    dist_coeffs = dist

    ignore_flags = {
        'ignore_tangential_distortion': cv2.CALIB_ZERO_TANGENT_DIST,
        'ignore_k3': cv2.CALIB_FIX_K3,
        'ignore_k2': cv2.CALIB_FIX_K2,
        'ignore_k1': cv2.CALIB_FIX_K1
    }

    for k, v in ignore_flags.items():
        flags += v
        retval, mat, dist, rvecs, tvecs, \
        std_intrinsics, std_extrinsics, errors = calibrate(imageSize=img_size, cameraMatrix=K, distCoeffs=D, flags=flags)
        ctx.logger(json.dumps({
            'ignore_type': k,
            're-project-error': retval, 
            'mat': str(mat.round(3).tolist()),
            'dist': str(dist.round(3).tolist())}, indent=4))
        if retval > error_threshold:
            break
        camera_matrix = mat
        dist_coeffs = dist
    
    return camera_matrix, dist_coeffs

def is_rotation_matrix(R):
    shouldBeIdentity = np.dot(np.transpose(R), R)
    return np.linalg.norm(np.identity(3, dtype=R.dtype) - shouldBeIdentity) < 1e-6

def rotation_matrix2euler_angles(R):
    assert(is_rotation_matrix(R))
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6
    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0
    return np.array([x, y, z])

def euler_angles2rotation_matrix(theta):
    # Rotation on x-axis
    r_pitch = np.array([
        [1, 0,                  0],
        [0, math.cos(theta[0]), -math.sin(theta[0])],
        [0, math.sin(theta[0]), math.cos(theta[0])]
    ])

    # Rotation on y-axis
    r_yaw = np.array([
        [math.cos(theta[1]),  0, math.sin(theta[1])],
        [0,                   1, 0],
        [-math.sin(theta[1]), 0, math.cos(theta[1])]
    ])

    # Rotation on z-axis
    r_roll = np.array([
        [math.cos(theta[2]), -math.sin(theta[2]), 0],
        [math.sin(theta[2]), math.cos(theta[2]),  0],
        [0,                  0,                   1]
    ])
    return np.dot(r_roll, np.dot(r_yaw, r_pitch))

def pixel_to_world(intrinsics_matrix, extrinsics_matrix, x, y):
    pseudo_inv_extrinsics = np.linalg.pinv(extrinsics_matrix)
    intrinsics_inv = np.linalg.inv(intrinsics_matrix)
    pixels_matrix = np.array((x, y, 1))
    ans = np.matmul(intrinsics_inv, pixels_matrix) # intrinsics_inv @ pixels_matrix
    ans = np.matmul(pseudo_inv_extrinsics, ans)  # pseudo_inv_extrinsics @ ans
    ans /= ans[-1] 
    return ans

def measure_by_point(camera_matrix, extrinsics_matrix, point):
    tl, tr, br, bl = point
    p1 = pixel_to_world(camera_matrix, extrinsics_matrix, tl[0], tl[1])
    p2 = pixel_to_world(camera_matrix, extrinsics_matrix, tr[0], tr[1])
    p3 = pixel_to_world(camera_matrix, extrinsics_matrix, bl[0], bl[1])
    W, H = p1 - p2, p1 - p3
    width = np.sqrt(W[0]**2 + W[1]**2)
    height = np.sqrt(H[0]**2 + H[1]**2)   
    return width, height

def single_aruco_estimate_pose(ctx, image, aruco_dict, camera_matrix, dist_coeffs):
    # marker_length = 1.0 * 26
    # aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_250)
    marker_corners, marker_ids, _ = cv2.aruco.detectMarkers(image, aruco_dict)
    if np.all(marker_ids is not None):
        for (marker_corner, marker_id) in zip(marker_corners, marker_ids):
            # corner: [[[306. 237.]
            #          [440. 218.]
            #          [477. 359.]
            #          [344. 389.]]]
            marker_length = 1.0 * (marker_id[0] - 60) # unit: mm
            if marker_length < 0:
                continue
            rvecs, tvecs, marker_points = cv2.aruco.estimatePoseSingleMarkers(marker_corner, marker_length, camera_matrix, dist_coeffs)

            # calculate the height/width (2D to 3D)
            (tl, tr, br, bl) = [(int(x[0]), int(x[1])) for x in marker_corner.reshape((4, 2))]

            # X (red color), Y (green color), Z (blue color)
            cv2.drawFrameAxes(image, camera_matrix, dist_coeffs, rvecs, tvecs, 10)  # Draw Axis

            # rvecs: [[[ 0.18987504  2.79373027 -0.9026566 ]]]
            # tvecs: [[[ 8.25180397  2.28427923 39.30899197]]]
            # rotation_matrix: [[-0.97191346,  0.18216257,  0.14900011],
            #                   [ 0.06054155,  0.80535491, -0.5896933 ],
            #                   [-0.22741802, -0.56411016, -0.79376368]]
            rotation_matrix, _ = cv2.Rodrigues(rvecs[0])
            extrinsics_matrix = np.concatenate([rotation_matrix, tvecs[0].T], axis=1) 
            width, height = measure_by_point(camera_matrix, extrinsics_matrix, marker_corner[0])
            distance = np.linalg.norm(tvecs[0])
            # ctx.logger('width: {}, height: {}, distance: {}'.format(width, height, distance))
            # angles = cv2.decomposeProjectionMatrix(extrinsics_matrix)[6].flatten()
            angles = rotation_matrix2euler_angles(rotation_matrix)
            roll, pitch, yaw = [math.degrees(angle) for angle in angles]
            cv2.putText(image,
                        'size {} x {} D:{}'.format(round(width, 1), round(height, 1), round(distance, 1)),
                        (tl[0], tl[1] + 15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS.BLUE, 1)
            cv2.putText(image,
                        'roll:{} pitch:{} yaw:{}'.format(round(roll, 1), round(pitch, 1), round(yaw, 1)),
                        (bl[0], bl[1] - 15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS.BLUE, 1)
            
            if ctx._aruco_tags_info:
                grid_extrinsics_matrix = ctx._aruco_tags_info.get_grid_extrinsics()
                gr_width, gr_height = measure_by_point(camera_matrix, grid_extrinsics_matrix, marker_corner[0])
                p1 = pixel_to_world(camera_matrix, ctx._aruco_tags_info.get_nearest_extrinsics(tl[0], tl[1]), tl[0], tl[1])
                p2 = pixel_to_world(camera_matrix, ctx._aruco_tags_info.get_nearest_extrinsics(tr[0], tr[1]), tr[0], tr[1])
                p3 = pixel_to_world(camera_matrix, ctx._aruco_tags_info.get_nearest_extrinsics(bl[0], bl[1]), bl[0], bl[1])
                W, H = p1 - p2, p1 - p3
                width = round(np.sqrt(W[0]**2 + W[1]**2), 1)
                height = round(np.sqrt(H[0]**2 + H[1]**2), 1)
                cv2.putText(image,
                            'size {} x {} vs {} x {}'.format(round(gr_width, 1), round(gr_height, 1), width, height),
                            (tl[0], tl[1] - 15),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS.BLUE, 1)
            break # test only one
        cv2.aruco.drawDetectedMarkers(image, marker_corners, marker_ids, borderColor=COLORS.CHOCOLATE)
    return image

def girdaruco_estimate_poses(ctx, image, board, aruco_dict, camera_matrix, dist_coeffs):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    marker_corners, marker_ids, rejected_points = cv2.aruco.detectMarkers(img_gray, aruco_dict)
    # Refine detected markers
    # Eliminates markers not part of our board, adds missing markers to the board
    marker_corners, marker_ids, rejectedImgPoints, recoveredIds = cv2.aruco.refineDetectedMarkers(
            image = img_gray,
            board = board,
            detectedCorners = marker_corners,
            detectedIds = marker_ids,
            rejectedCorners = rejected_points,
            cameraMatrix = camera_matrix,
            distCoeffs = dist_coeffs)
    image = cv2.aruco.drawDetectedMarkers(image, marker_corners, borderColor=COLORS.CHOCOLATE)
    if marker_ids is None or len(marker_ids) < 20:
        raise
    # Estimate the posture of the gridboard, which is a construction of 3D space
    pose, rvec, tvec = cv2.aruco.estimatePoseBoard(marker_corners, marker_ids, board, camera_matrix, dist_coeffs, None, None)
    if pose:
        # Draw the camera posture calculated from the gridboard
        image = cv2.drawFrameAxes(image, camera_matrix, dist_coeffs, rvec, tvec, 20)
    return image, marker_corners, marker_ids, rvec, tvec

class ArucoTagsInfo(object):
            
    def __init__(self, grid_extrinsics_matrix, cx_map, cy_map, gap):
        self._grid_extrinsics_matrix = grid_extrinsics_matrix
        self._gap = gap
        self._cx_map_ordered = collections.OrderedDict(sorted(cx_map.items(), key=lambda kv: kv[0]))
        self._cy_map_ordered = collections.OrderedDict(sorted(cy_map.items(), key=lambda kv: kv[0]))
        self._cx_keys_list = list(self._cx_map_ordered.keys())
        self._cy_keys_list = list(self._cy_map_ordered.keys())
        
    def get_grid_extrinsics(self):
        return self._grid_extrinsics_matrix 
    
    def get_nearest_extrinsics(self, x, y):
        x_min, x_max = x - self._gap, x + self._gap
        y_min, y_max = y - self._gap, y + self._gap
        setx, sety = set(), set()
        for i, key in enumerate(self._cx_keys_list):
            if key > x_min and key < x_max:
                setx.add(self._cx_map_ordered[key])
        for i, key in enumerate(self._cy_keys_list):
            if key > y_min and key < y_max:
                sety.add(self._cy_map_ordered[key])
        balls = list(setx.intersection(sety))
        if len(balls) == 0:
            return self._grid_extrinsics_matrix
        if len(balls) == 1:
            return balls[0]._e
        k, b = 2 * self._gap, balls[0]
        for ball in balls[1:]:
            d = abs(ball._x - x) + abs(ball._y - y)
            if d < k:
                k, b = d, ball
        return b._e
                 
def detect_object_and_measure(ctx, image, dettype, camera_matrix, dist_coeffs):
    result = {}
    if dettype == 'aruco-6x6': # Detect 6x6 ArUco Marker
        dict_id = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_1000)
        result['aruco'] = single_aruco_estimate_pose(ctx, image, dict_id, camera_matrix, dist_coeffs)
    elif dettype == 'grid-26mm-3mm-9x7-4x4':
        markersX, markersY, marker_size_mm, gap_size_mm = 9, 7, 26, 3
        axis_length = int(0.5 * marker_size_mm)
        dict_id = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_250)
        board = cv2.aruco.GridBoard_create(
            markersX=markersX,
            markersY=markersY,
            markerLength=marker_size_mm,
            markerSeparation=gap_size_mm,
            dictionary=dict_id)
        try:
            class _Ball(object):
                def __init__(self, x, y, e):
                    self._x = x
                    self._y = y
                    self._e = e
            img_gird_pose, marker_corners, marker_ids, rvec, tvec = girdaruco_estimate_poses(ctx, image.copy(), board, dict_id, camera_matrix, dist_coeffs)
            result['grid_pose'] = img_gird_pose
            if len(marker_corners) == markersX * markersY:
                grid_extrinsics_matrix = np.concatenate([cv2.Rodrigues(rvec)[0], tvec], axis=1) 
                rvecs, tvecs, marker_points = cv2.aruco.estimatePoseSingleMarkers(marker_corners, marker_size_mm, camera_matrix, dist_coeffs)
                xs_map, ys_map, length = {}, {}, -1
                for mc, mid, rvec, tvec in zip(marker_corners, marker_ids, rvecs, tvecs):
                    extrinsics_matrix = np.concatenate([cv2.Rodrigues(rvec)[0], tvec.T], axis=1) 
                    if length < 0:
                        length = max(abs(mc[0][0][0] - mc[0][2][0]), abs(mc[0][0][1] - mc[0][2][1]))
                    center = np.mean(mc[0], axis=0, dtype=np.int32).tolist()
                    ball = _Ball(center[0], center[1], extrinsics_matrix)
                    xs_map[center[0]], ys_map[center[1]] = ball, ball
                    width, _ = measure_by_point(camera_matrix, extrinsics_matrix, mc[0])
                    cv2.putText(image, f'{mid} {round(width, 1)}', (center[0] - 5, center[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.3, COLORS.BLUE, 1)                
                    cv2.drawFrameAxes(image, camera_matrix, dist_coeffs, rvec, tvec, axis_length)
                ctx._aruco_tags_info = ArucoTagsInfo(grid_extrinsics_matrix, xs_map, ys_map, length)
            result['markers'] = image
        except Exception as err:
            ctx.logger(f'{err}: {traceback.format_exc()}')
        
    elif dettype == 'black_edge_carton':
        if ctx._aruco_tags_info is None:
            cv2.putText(image, 'no extrinsics matrix', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS.LIGHT_BLUE, 1)
        
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
        img_edged = cv2.Canny(img_blur, 50, 100)
        img_edged = cv2.dilate(img_edged, None, iterations=1)
        img_edged = cv2.erode(img_edged, None, iterations=1)
        contours = cv2.findContours(img_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        # cv2.drawContours(image, contours, -1, color=(0, 0, 255), thickness=4)
        # for cnt in contours:
        #     area = cv2.contourArea(cnt)
        #     if area > 1000 and area < 20000:
        #         rect = cv2.minAreaRect(cnt)
        #         (x, y), (w, h), angle = rect
        #         box = cv2.boxPoints(rect)
        #         box = np.int0(box)
        #         cv2.circle(image, (int(x), int(y)), 10, (0, 0, 255), -1)
        #         cv2.polylines(image, [box], True, (100, 10, 0), 2)
    return result

### Calibration By ChessBoard

In [30]:
def start_calibrate_by_chessboard(
    ctx, w_btn_source, squares_x, squares_y, square_size, win_size, term_iters, term_eps,
    sample_size, use_file, w_det_objtype, w_sample_list, w_cam_frame):
    
    ctx.logger(f'start: {squares_x}, {squares_y}, {square_size}mm, {sample_size}', clear=1)
    
    # Object points for a chessboard
    pattern_size = (squares_x - 1, squares_y - 1)
    objp = np.zeros((np.prod(pattern_size), 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
    objp = objp * square_size
        
    win_size, zero_zone = (win_size, win_size), (-1, -1)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, term_iters, term_eps)

    def _video_capture():
        w_btn_source.disabled = True
        out_dir = f'out/chessboard/{squares_x}x{squares_y}_{square_size}'
        try:
            obj_points, img_points = [], []
            samples = []
            options = [('Camera', 0)]
            if sample_size is None: 
                image_list = glob.glob(f'{out_dir}/*.png')
                display_images = {}
                for path in image_list:
                    name = os.path.basename(path)
                    img = cv2.imread(path)
                    # img = cv2.flip(img, 1)
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    found, corners = cv2.findChessboardCorners(gray, pattern_size)
                    better_chess_corners = cv2.cornerSubPix(gray, corners, win_size, zero_zone, criteria)
                    obj_points.append(objp)
                    img_points.append(better_chess_corners)
                    display_images['orig_corners'] = cv2.drawChessboardCorners(img.copy(), pattern_size, corners, found)
                    display_images['best_corners'] = cv2.drawChessboardCorners(img.copy(), pattern_size, better_chess_corners, found)
                    options.append((name, int(name.split('.')[0])))
                    samples.append(nbeasy_widget_display(display_images))
                w_sample_list.options = sorted(options, key=lambda item: item[1])
                state = CalibrateState.CALIBRATE
            else:
                shutil.rmtree(out_dir, ignore_errors=True)
                state = CalibrateState.COLLECT
            os.makedirs(out_dir, exist_ok=True)
                
            camera = cv2.VideoCapture(0)
            frame_width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
            frame_irate = int(camera.get(cv2.CAP_PROP_FPS))
            
            ctx.logger(f'frame_width:{frame_width} frame_height:{frame_height} display: {w_cam_frame.width}')

            ctx.camera = camera
            ctx.camera_is_running = camera.isOpened()

            ctx._aruco_tags_info = None
            camera_matrix, dist_coeffs = None, None

            frame_idx, iter_idx = 0, 0
            while ctx.camera_is_running:
                if w_sample_list.value != 0:
                    w_cam_frame.value = samples[w_sample_list.value - 1]
                    time.sleep(1)
                    continue
                ret, frame_bgr = camera.read()
                if not ret:
                    break
                # frame_bgr = cv2.flip(frame_bgr, 1)
                display_frames = {'raw': frame_bgr}
                frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
                if state == CalibrateState.COLLECT:
                    found, corners = cv2.findChessboardCorners(frame_gray, pattern_size)
                    if found:
                        # Find better sub pix position for the corners in the roof corners neighbourhood
                        better_chess_corners = cv2.cornerSubPix(frame_gray, corners, win_size, zero_zone, criteria)
                        display_frames['orig_corners'] = cv2.drawChessboardCorners(frame_bgr.copy(), pattern_size, corners, found)
                        display_frames['best_corners'] = cv2.drawChessboardCorners(frame_bgr.copy(), pattern_size, better_chess_corners, found)

                        if frame_idx % frame_irate == 0:
                            obj_points.append(objp)
                            img_points.append(better_chess_corners)
                            iter_idx += 1
                            
                            box = cv2.boxPoints(cv2.minAreaRect(better_chess_corners.reshape(-1, 2)))
                            cv2.drawContours(display_frames['best_corners'], [np.int0(box)], 0, COLORS.ORANGE, 2)
                            
                            # Draw and display the corners
                            options.append((str(iter_idx) + '.png', iter_idx))
                            samples.append(io.BytesIO(cv2.imencode('.png', display_frames['best_corners'])[1]).getvalue())
                            if iter_idx == sample_size:
                                state = CalibrateState.CALIBRATE
                                w_sample_list.options = sorted(options, key=lambda item: item[1])
                            cv2.imwrite(f'{out_dir}/{iter_idx}.png', frame_bgr)
                    cv2.putText(display_frames['best_corners'], f'Count: {iter_idx}', (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1.2, COLORS.LIGHT_BLUE, 2)
                elif state == CalibrateState.CALIBRATE and not use_file:
                    camera_matrix, dist_coeffs = start_calibrate_camera_with_good_flags(
                        ctx, functools.partial(
                            cv2.calibrateCameraExtended, objectPoints=obj_points, imagePoints=img_points),
                        img_size=frame_gray.shape[::-1],
                    )

                    save_calibration_intrinsics(camera_matrix, dist_coeffs, out_dir)
                    
                    # obj_points, img_points = obj_points[-1], img_points[-1]
                    # # Calculating extrinsics by last imgage (rotation vector)
                    # retval, rvec, tvec = cv2.solvePnP(obj_points, img_points, camera_matrix, dist_coeffs)                    
                    # # rvec, tvec = np.mean(np.array(rvecs), axis=0), np.mean(np.array(tvecs), axis=0)
                    # # transform rotation vector to ratation matrix
                    # rotation_matrix, _ = cv2.Rodrigues(rvec)
                    # extrinsics_matrix = np.concatenate([rotation_matrix, tvec], 1) 
                    
                    # calculate the re-project points error.  == reval
                    # mean_error = 0
                    # for obj_point, img_point, rvec, tvec in zip(obj_points, img_points, rvecs, tvecs):
                    #     img_point_proj, _ = cv2.projectPoints(obj_point, rvec, tvec, camera_matrix, dist_coeffs)
                    #     error = cv2.norm(img_point, img_point_proj, cv2.NORM_L2) / len(img_point_proj)
                    #     mean_error += error
                    # ctx.logger('total error: {}'.format(mean_error / len(obj_points)))
             
                    state = CalibrateState.COMPLETED
                else: 
                    # load from file
                    if camera_matrix is None and use_file:
                        camera_matrix, dist_coeffs = load_calibration_intrinsics(out_dir)
                        
                    display_frames['undist'] = cv2.undistort(frame_bgr.copy(), camera_matrix, dist_coeffs)
                    result = detect_object_and_measure(ctx, frame_bgr.copy(), w_det_objtype.value, camera_matrix, dist_coeffs)
                    for title, image in result.items():
                        display_frames[title] = image

                # display image
                nbeasy_widget_display(display_frames, w_cam_frame)
                frame_idx += 1

        except Exception as err:
            ctx.logger(f'{err}: {traceback.format_exc()}')
        finally:
            w_btn_source.disabled = False
            ctx.camera.release()
            ctx.camera_is_running = False
            ctx.camera = None

    camera_thread = threading.Thread(target=_video_capture, name='camera')
    camera_thread.daemon = True
    camera_thread.start()

stop_calibrate_by_chessboard = stop_camera_calibrate

In [10]:
def start_calibrate_by_charucoboard(
    ctx, w_btn_source, squares_x, squares_y, square_size, marker_size, win_size, term_iters, term_eps,
    aruco_id, sample_size, use_file, w_det_objtype, w_sample_list, w_cam_frame):
    
    ctx.logger(f'start: {squares_x}, {squares_y}, {square_size}mm, {marker_size}mm, {aruco_id}, {sample_size}', clear=1)
    
    charuco_dict = cv2.aruco.getPredefinedDictionary(aruco_id)
    charuco_board = cv2.aruco.CharucoBoard_create(squares_x, squares_y, 0.001 * square_size, 0.001 * marker_size, charuco_dict)
    
    pattern_size = (squares_x, squares_y)
    win_size, zero_zone = (win_size, win_size), (-1, -1)
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, term_iters, term_eps)
        
    def _detect_charuco_corners(frame_gray, charuco_board, charuco_dict, win_size, zero_zone, criteria):
        # corner detection
        marker_corners, marker_ids, rejected_points = cv2.aruco.detectMarkers(frame_gray, charuco_dict)
        if marker_corners is not None and len(marker_corners) > 0:
            # corner refine 
            # marker_corners, marker_ids, refusd, recoverd = cv2.aruco.refineDetectedMarkers(
            #     frame_gray, charuco_board, marker_corners, marker_ids, rejectedCorners=rejected_points) 
            # corner interpolation (get charuco corners and ids from detected aruco markers)
            retval, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(marker_corners, marker_ids, frame_gray, charuco_board)
            charuco_corners is not None and len(charuco_corners) > 5
            if charuco_corners is not None and len(charuco_corners) > 5:
                if win_size:
                    better_charuco_corners = cv2.cornerSubPix(frame_gray, charuco_corners, win_size, zero_zone, criteria)
                else:
                    better_charuco_corners  = charuco_corners
                return retval, marker_corners, marker_ids, charuco_corners, better_charuco_corners, charuco_ids 
        return None, None, None, None, None, None
    
    def _video_capture():
        w_btn_source.disabled = True
        result = {}
        obj_points, img_points = [], []
        all_charuco_corners, all_charuco_ids = [], []
        out_dir = f'out/chessaruco/{squares_x}x{squares_y}_{square_size}_{marker_size}'
            
        try:
            samples = []
            options = [('Camera', 0)]
            if sample_size is None: 
                image_list = glob.glob(f'{out_dir}/*.png')
                for path in image_list:
                    name = os.path.basename(path)
                    img = cv2.imread(path)
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    retval, marker_corners, marker_ids, charuco_corners, better_charuco_corners, charuco_ids = _detect_charuco_corners(
                            gray, charuco_board, charuco_dict, win_size, zero_zone, criteria)
                    if retval:
                        all_charuco_corners.append(better_charuco_corners)
                        all_charuco_ids.append(charuco_ids)
                        cv2.aruco.drawDetectedCornersCharuco(img, better_charuco_corners, charuco_ids)
                        options.append((name, int(name.split('.')[0])))
                        samples.append(io.BytesIO(cv2.imencode('.png', img)[1]).getvalue())
                w_sample_list.options = sorted(options, key=lambda item: item[1])
                state = CalibrateState.CALIBRATE
            else:
                shutil.rmtree(out_dir, ignore_errors=True)
                state = CalibrateState.COLLECT
                
            os.makedirs(out_dir, exist_ok=True)
                
            camera = cv2.VideoCapture(0)
            frame_width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
            frame_irate = int(camera.get(cv2.CAP_PROP_FPS))
            
            ctx.logger(f'frame_width:{frame_width} frame_height:{frame_height} display: {w_cam_frame.width}')

            ctx.camera = camera
            ctx.camera_is_running = camera.isOpened()
            
            ctx._aruco_tags_info = None
            camera_matrix, dist_coeffs = None, None,
            
            frame_idx, iter_idx = 0, 0
            
            while ctx.camera_is_running:
                if w_sample_list.value != 0:
                    if w_cam_frame.layout.width != frame_width:
                        w_cam_frame.layout.width = f'{frame_width}px'
                        w_cam_frame.layout.height = f'{frame_height}px'
                    w_cam_frame.value = samples[w_sample_list.value]
                    time.sleep(1)
                    continue
                ret, frame_bgr = camera.read()
                if not ret:
                    break
                # frame_bgr = cv2.flip(frame_bgr, 1)
                display_frames = {'raw': frame_bgr}
                frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
                if state == CalibrateState.COLLECT:
                    retval, marker_corners, marker_ids, charuco_corners, better_charuco_corners, charuco_ids = _detect_charuco_corners(
                        frame_gray, charuco_board, charuco_dict, win_size, zero_zone, criteria)
                    
                    if retval:
                        display_frames['markers'] = cv2.aruco.drawDetectedMarkers(frame_bgr.copy(), marker_corners, marker_ids)                        
                        display_frames['orig_corners'] = cv2.aruco.drawDetectedCornersCharuco(frame_bgr.copy(), charuco_corners, charuco_ids)
                        display_frames['best_corners'] = cv2.aruco.drawDetectedCornersCharuco(frame_bgr.copy(), better_charuco_corners, charuco_ids)

                        if frame_idx % frame_irate == 0:
                            # Draw and Display markers and corners
                            # objp = charuco_board.chessboardCorners[all_charuco_ids.flatten()]
                            all_charuco_corners.append(better_charuco_corners)
                            all_charuco_ids.append(charuco_ids)
                            iter_idx += 1

                            box = cv2.boxPoints(cv2.minAreaRect(better_charuco_corners.reshape(-1, 2)))
                            cv2.drawContours(display_frames['best_corners'], [np.int0(box)], 0, COLORS.ORANGE, 2)

                            options.append((str(iter_idx) + '.png', iter_idx))
                            samples.append(io.BytesIO(cv2.imencode('.png', display_frames['best_corners'])[1]).getvalue())
                            if iter_idx == sample_size:
                                state = CalibrateState.CALIBRATE
                                w_sample_list.options = sorted(options, key=lambda item: item[1])
                            cv2.imwrite(f'{out_dir}/{iter_idx}.png', frame_bgr)
                        cv2.putText(display_frames['best_corners'], f'Count: {iter_idx}', (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1.2, COLORS.GOLDEN, 2)
                elif state == CalibrateState.CALIBRATE and not use_file:
                    camera_matrix, dist_coeffs = start_calibrate_camera_with_good_flags(
                        ctx, functools.partial(
                            cv2.aruco.calibrateCameraCharucoExtended,
                            charucoCorners=all_charuco_corners, charucoIds=all_charuco_ids,
                            board=charuco_board),
                        img_size=frame_gray.shape[::-1]
                    )
                        
                    save_calibration_intrinsics(camera_matrix, dist_coeffs, out_dir)
                    
                    # total_error = 0.0
                    # for k, (charuco_corners, c_ids, rvec, tvec) in enumerate(zip(all_charuco_corners, all_charuco_ids, rvecs, tvecs)):
                    #     obj_points, img_points = cv2.aruco.getBoardObjectAndImagePoints(charuco_board, charuco_corners, c_ids)
                    #     prj_points, _ = cv2.projectPoints(obj_points, rvec, tvec, camera_matrix, dist_coeffs)
                    #     error = cv2.norm(img_points, prj_points, cv2.NORM_L2) / len(obj_points)
                    #     total_error += error
                    # ctx.logger(f'mean total error: {total_error / len(all_charuco_corners)}')
                    state = CalibrateState.COMPLETED
                else: 
                    # load from file
                    if camera_matrix is None and use_file:
                        camera_matrix, dist_coeffs = load_calibration_intrinsics(out_dir)
                                                
                    display_frames['undist'] = cv2.undistort(frame_bgr.copy(), camera_matrix, dist_coeffs)
                    result = detect_object_and_measure(ctx, frame_bgr.copy(), w_det_objtype.value, camera_matrix, dist_coeffs)
                    for title, image in result.items():
                        display_frames[title] = image
                    
                # display image
                nbeasy_widget_display(display_frames, w_cam_frame)
                frame_idx += 1
                
        except Exception as err:
            ctx.logger(f'{err}: {traceback.format_exc()}')
        finally:
            w_btn_source.disabled = False
            ctx.camera.release()
            ctx.camera_is_running = False
            ctx.camera = None
        
    camera_thread = threading.Thread(target=_video_capture, name='camera')
    camera_thread.daemon = True
    camera_thread.start()

stop_calibrate_by_charucoboard = stop_camera_calibrate

### Template Schema

In [31]:
def nbeasy_widget_display(images, img_wid=None):
    C = len(images)
    show_ncol, show_nrow = 1, 1
    if C > 1:
        show_ncol = 2
        for i in range(C % show_ncol):
            images[f'placehold-{i}'] = images[list(images.keys())[-1]].copy() # 255 * np.ones_like(images[list(images.keys())[0]])
        show_nrow = len(images) // show_ncol
        row_images = []
        col_images = []
        for key, img in images.items():
            cv2.putText(img, key, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS.LIGHT_BLUE, 1)
            col_images.append(img)
            if len(col_images) == show_ncol:
                row_images.append(np.hstack(col_images))
                col_images = []
                
        display_image = np.vstack(row_images)
    else:
        display_image = images.popitem()[1]
    
    imgbytes = io.BytesIO(cv2.imencode('.png', display_image)[1]).getvalue()
    if img_wid:
        img_wid.layout.width = f'{int(img_wid.width) * show_ncol}px'
        img_wid.layout.height = f'{int(img_wid.height) * show_nrow}px'
        img_wid.value = imgbytes
    else:
        return imgbytes
    
def nbeasy_chessboard_choice_objs(nrow, ncol, square_size, width=300, ro=True):
    return {
        'type': 'H',
        'objs': [
            nbeasy_widget_int('cfg.chessboard_squares_x', 'Squares X', nrow, width=width, readonly=ro),
            nbeasy_widget_int('cfg.chessboard_squares_y', 'Squares Y', ncol, width=width, readonly=ro),
            nbeasy_widget_int('cfg.chessboard_square_size', 'Square Size(mm)', square_size, width=width, readonly=ro)        
        ]
    }

def nbeasy_charucoboard_choice_objs(nrow, ncol, square_size, marker_size, aruco_dict_id, width=300, ro=True):
    enums = [
        ('4x4', cv2.aruco.DICT_4X4_1000),
        ('5x5', cv2.aruco.DICT_5X5_1000),
    ]
    return {
        'type': 'H',
        'objs': [
            nbeasy_widget_int('cfg.charucoboard_squares_x', 'Squares X', nrow, width=width, readonly=ro),
            nbeasy_widget_int('cfg.charucoboard_squares_y', 'Squares Y', ncol, width=width, readonly=ro),
            nbeasy_widget_int('cfg.charucoboard_square_size', 'Square Size(mm)', square_size, width=width, readonly=ro),
            nbeasy_widget_float('cfg.charucoboard_marker_size', 'Marker Size CM', marker_size, width=width, readonly=ro),
            nbeasy_widget_stringenum(
                'cfg.charucoboard_aruco_dict', 'ArUco Dict', default=aruco_dict_id,
                enums=enums,
                width=width,
                readonly=ro
            )
        ]
    }

def nbeasy_common_shared_by_tagtype(tag):
    return {
        'type': 'V',
        'objs': [
            {
                'type': 'H',
                'objs': [
                    nbeasy_widget_int(f'cfg.{tag}_win_size', 'Win Size', default=5),
                    nbeasy_widget_int(f'cfg.{tag}_term_iters', 'Term Iters', default=500),
                    nbeasy_widget_float(f'cfg.{tag}_term_eps', 'Term EPS', default=0.0001),
                ]
            },
            nbeasy_widget_booltrigger(f'_cfg.{tag}_is_online', 'Online', default=False, triggers=[
                {
                    'objs': [
                        nbeasy_widget_bool(f'cfg.{tag}_use_file', 'Use File', default=True)
                    ]
                },
                {
                    'objs': [
                        nbeasy_widget_int(f'cfg.{tag}_sample_size', 'Sample Size', 32, width=333)
                    ]
                }
            ], width=200),
            {
                'type': 'V',
                'name': 'Camera',
                'objs': [
                    {
                        'type': 'H',
                        'objs': [
                            nbeasy_widget_button(f'cfg.__btn_{tag}_start_camera', 'Start', width=200, style='success'),
                            nbeasy_widget_button(f'cfg.__btn_{tag}_stop_camera', 'Stop', width=200, style='success'),
                        ],
                        'justify_content': 'center'
                    },
                ],
                'align_items': 'center'
            },
        ]
    }

schema = {
    'type': 'page',
    'objs': [
        {
            'type': 'tab',
            'objs': [
                {
                    'type': 'V',
                    'name': 'ChessBoard',
                    'objs': [
                        nbeasy_widget_stringenumtrigger(
                            '_cfg.chessboard_choice', 'Board Choise', default=1,
                            enums = ['A4-15mm-11x8', 'A4-30mm-9x7', 'A3-25mm-16x11'],
                            triggers = [
                                nbeasy_chessboard_choice_objs(11, 8, 15),
                                nbeasy_chessboard_choice_objs(9, 7, 30),
                                nbeasy_chessboard_choice_objs(16, 11, 25),
                            ],
                            width=333),
                        nbeasy_common_shared_by_tagtype('chessboard'),
                    ],
                }, # end chessboard
                {
                    'type': 'V',
                    'name': 'CharucoBoard',
                    'objs': [
                        nbeasy_widget_stringenumtrigger(
                            '_cfg.charucoboard_choice', 'Board Choise', default=1,
                            enums = ['A3-20mm-16mm-19x13-4x4', 'A4-30mm-23mm-9x6-4x4', 'A4-25mm-19mm-11x8-5x5', 'A4-15mm-12mm-11x8-5x5'],
                            triggers = [
                                nbeasy_charucoboard_choice_objs(19, 13, 20, 16, 0),
                                nbeasy_charucoboard_choice_objs(9, 6, 30, 23, 0),
                                nbeasy_charucoboard_choice_objs(11, 8, 25, 19, 1),
                                nbeasy_charucoboard_choice_objs(11, 8, 15, 12, 1),
                            ],
                            width=333),
                        nbeasy_common_shared_by_tagtype('charucoboard'),
                    ],
                }, # end charucoboard
            ], # end tab objs
        },
        {
            'type': 'V',
            'objs': [
                nbeasy_widget_image('__cfg.video_frame', 'Frame', '', width=640, height=480),
                {
                    'type': 'H',
                    'objs': [
                        nbeasy_widget_stringenum(
                            'cfg.det_object_type', 'Detect Object', default=0,
                            enums=['aruco-6x6', 'grid-26mm-3mm-9x7-4x4', 'black_edge_carton'],
                            width=300),
                        nbeasy_widget_stringenum(
                            'cfg.sample_list', 'Sample List', default=0,
                            enums=[('Camera', 0)],
                            width=300)
                    ],
                    'justify_content': 'center'
                }
            ],
            'align_items': 'center'
        }, # end video
    ], # end page objs
    'evts': [
        {
            'type': 'jsdlink',
            'objs': [
                {
                    'source': 'cfg.__btn_chessboard_start_camera:disabled',
                    'target': 'cfg.__btn_charucoboard_start_camera:disabled'
                }
            ]
        },
        {
            'type': 'onclick',
            'objs': [
                {
                    'handler': start_calibrate_by_chessboard,
                    'params': {
                        'sources': ['cfg.__btn_chessboard_start_camera'],
                        'targets': [
                            'cfg.chessboard_squares_x:value',
                            'cfg.chessboard_squares_y:value',
                            'cfg.chessboard_square_size:value',
                            'cfg.chessboard_win_size:value',
                            'cfg.chessboard_term_iters:value',
                            'cfg.chessboard_term_eps:value',
                            'cfg.chessboard_sample_size:value',
                            'cfg.chessboard_use_file:value',
                            'cfg.det_object_type',
                            'cfg.sample_list',
                            '__cfg.video_frame'
                        ]
                    }
                },
                {
                    'handler': stop_calibrate_by_chessboard,
                    'params': {
                        'sources': ['cfg.__btn_chessboard_stop_camera'],
                        'targets': [
                        ]
                    }
                },
                {
                    'handler': start_calibrate_by_charucoboard,
                    'params': {
                        'sources': ['cfg.__btn_charucoboard_start_camera'],
                        'targets': [
                            'cfg.charucoboard_squares_x:value',
                            'cfg.charucoboard_squares_y:value',
                            'cfg.charucoboard_square_size:value',
                            'cfg.charucoboard_marker_size:value',
                            'cfg.charucoboard_win_size:value',
                            'cfg.charucoboard_term_iters:value',
                            'cfg.charucoboard_term_eps:value',
                            'cfg.charucoboard_aruco_dict:value',
                            'cfg.charucoboard_sample_size:value',
                            'cfg.charucoboard_use_file:value',
                            'cfg.det_object_type',
                            'cfg.sample_list',
                            '__cfg.video_frame'
                        ]
                    }
                },
                {
                    'handler': stop_calibrate_by_charucoboard,
                    'params': {
                        'sources': ['cfg.__btn_charucoboard_stop_camera'],
                        'targets': [
                        ]
                    }
                },
            ]
        }
    ], # end event
}

if g_ctx:
    stop_camera_calibrate(g_ctx)
g_ctx = nbeasy_schema_parse(schema, debug=True)

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(VBox(children=(Dropdown(description='B…

In [ ]:
raise

## UnD-1

In [ ]:
def nbon_start_camera(ctx, sbtn,
                      w_bgsub_method, w_bgsub_history, w_det_shadow, 
                      w_rmn_shadow_threshold, w_rmn_morpho_kernel, rmn_erode_iter, rmn_dilate_iter,
                      w_cnt_area_threshold,
                      w_cam_frame):
    method = str.lower(w_bgsub_method.value)
    history_length = w_bgsub_history.value
    detect_shadows = w_det_shadow.value
    
    shadow_thresh = w_rmn_shadow_threshold.value
    kernel = np.ones((w_rmn_morpho_kernel.value, w_rmn_morpho_kernel.value), np.uint8)
    
    erode_iter = rmn_erode_iter.value
    dilate_iter = rmn_dilate_iter.value
    
    area_threshold = w_cnt_area_threshold.value
    
    ctx.logger(f'nbon_start_camera({method}, {history_length}, {detect_shadows}, {shadow_thresh}, {w_rmn_morpho_kernel.value})')
    
    import threading
    
    def _video_capture():
        sbtn.disabled = True
        parameters = cv2.aruco.DetectorParameters_create()
        aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_5X5_50)
        pixel_cm_ratio_1, pixel_cm_ratio_2 = -1, -1
        marker_corner_1, marker_corner_2 = None, None
        try:
            bg_object = None
            if method == 'knn':
                bg_object = cv2.createBackgroundSubtractorKNN(history=history_length, detectShadows=detect_shadows)
            elif method == 'mog2':
                bg_object = cv2.createBackgroundSubtractorMOG2(history=history_length, detectShadows=detect_shadows)

            camera = cv2.VideoCapture(0)
            width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))

            ctx.camera = camera
            ctx.camera_is_running = ctx.camera.isOpened()
            while ctx.camera_is_running:
                ret, frame_bgr = camera.read()
                if not ret:
                    break
                    
                # Detect Aruco markers 5cm x 5cm
                corners, ids, _ = cv2.aruco.detectMarkers(frame_bgr, aruco_dict, parameters=parameters)
                if corners is None or len(corners) == 0: 
                    w_cam_frame.value = io.BytesIO(cv2.imencode('.png', frame_bgr)[1]).getvalue()
                    continue
                    
                frame_copy = frame_bgr.copy()
                    
                for (marker_corner, marker_ID) in zip(corners, ids):
                    (tl, tr, br, bl) = [(int(x[0]), int(x[1])) for x in marker_corner.reshape((4, 2))]
                    rect = cv2.minAreaRect(marker_corner[0])
                    if marker_ID == 1:
                        pixel_cm_ratio_1 = cv2.arcLength(marker_corner, True) / (4 * 5) # place it on the target left
                        marker_corner_1 = (tl, tr, br, bl)
                        cv2.putText(frame_copy,
                                    'id:1 {}x{}'.format(round(rect[1][0], 2), round(rect[1][1], 2)),
                                    (marker_corner_1[0][0], marker_corner_1[0][1] - 15),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                    elif marker_ID == 2:
                        pixel_cm_ratio_2 = cv2.arcLength(marker_corner, True) / (4 * 5) # place it on the target right
                        marker_corner_2 = (tl, tr, br, bl)
                        cv2.putText(frame_copy,
                                    'id:2 {}x{}'.format(round(rect[1][0], 2), round(rect[1][1], 2)),
                                    (marker_corner_2[0][0], marker_corner_2[0][1] - 15),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                        
                if marker_corner_1 is None and marker_corner_2 is None:
                    w_cam_frame.value = io.BytesIO(cv2.imencode('.png', frame_bgr)[1]).getvalue()
                    continue
                                                             
                cv2.aruco.drawDetectedMarkers(frame_copy, corners, ids, borderColor=(255, 0, 0))

                if bg_object:
                    # Apply the background object on the frame to get the segmented mask.     
                    fgmask = bg_object.apply(frame_bgr)

                    # Perform thresholding to get rid of the shadows.
                    _, fgmask = cv2.threshold(fgmask, shadow_thresh, 255, cv2.THRESH_BINARY)

                    # Apply some morphological operations to make sure you have a good mask
                    fgmask = cv2.erode(fgmask, kernel, iterations=erode_iter)
                    fgmask = cv2.dilate(fgmask, kernel, iterations=dilate_iter)

                    # Get foreground object
                    foreground = cv2.bitwise_and(frame_bgr, frame_bgr, mask=fgmask)
                    
                    frame_mid = foreground
                    frame_dst = fgmask
                else:
                    # Grayscale & Guassian blur
                    frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY) 
                    frame_blur = cv2.GaussianBlur(frame_gray, (3, 3), 0)

                    # # Divide gray by morphology image
                    # frame_div = cv2.divide(frame_gray, frame_blur, scale=255)

                    # Sharpen using unsharp masking
                    # frame_sharp = filters.unsharp_mask(frame_div, radius=1.5, amount=1.5, channel_axis=None, preserve_range=False)
                    # frame_sharp = (255 * frame_sharp).clip(0, 255).astype(np.uint8)

                    # Otsu Filter
                    # _, frame_otsu = cv2.threshold(frame_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                    _, frame_otsu = cv2.threshold(frame_gray, 0, 255, cv2.THRESH_OTSU)

                    # Detect edge by candy
                    # frame_canny = cv2.Canny(frame_blur, 200, 250)
                    
                    frame_mid = cv2.cvtColor(frame_otsu, cv2.COLOR_GRAY2BGR)
                    frame_dst = frame_otsu

                contours, _ = cv2.findContours(frame_dst, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
                for cnt in contours:
                    if cv2.contourArea(cnt) > area_threshold:
                        x1, y1, width, height = cv2.boundingRect(cnt)
                        x2, y2 = x1 + width, y1 + height
                        
                        cv2.rectangle(frame_copy, (x1, y1), (x2, y2),(0, 0, 255), 2)
                        cv2.putText(frame_copy,
                                    '{}x{}'.format(round(width, 2), round(height, 2)),
                                    (x2, y2),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                        
                        if marker_corner_1:
                            if x1 < marker_corner_1[1][0] or x1 < marker_corner_1[2][0]:
                                continue
                        if marker_corner_2:
                            if x2 > marker_corner_2[0][0] or x2 > marker_corner_2[3][0]:
                                continue
                        
                        rect = cv2.minAreaRect(cnt)
                        (cx, cy), (cw, ch), angle = rect
                        
                        if marker_corner_1 and marker_corner_2:
                            ow = round(0.5 * cw / pixel_cm_ratio_1 + 0.5 * cw / pixel_cm_ratio_2, 2)
                            oh = round(0.5 * ch / pixel_cm_ratio_1 + 0.5 * ch / pixel_cm_ratio_2, 2)
                        elif marker_corner_1:
                            ow = round(cw / pixel_cm_ratio_1, 2)
                            oh = round(ch / pixel_cm_ratio_1, 2)
                        else:
                            ow = round(cw / pixel_cm_ratio_2, 2)
                            oh = round(ch / pixel_cm_ratio_2, 2)
                        
                        cv2.circle(frame_copy, (int(cx), int(cy)), 10, (255, 0, 0), -1)
                        cv2.polylines(frame_copy, [np.int0(cv2.boxPoints(rect))], True, (255, 0, 0), 2)
                        cv2.putText(frame_copy, "{}x{}".format(ow, oh), (int(cx - 100), int(cy - 20)), cv2.FONT_HERSHEY_PLAIN, 0.5, (100, 200, 0), 1)

                stacked = np.hstack((frame_bgr, frame_mid, frame_copy))
                w_cam_frame.value = io.BytesIO(cv2.imencode('.png', stacked)[1]).getvalue()
                
        except Exception as err:
            ctx.logger(f'{err}')
        finally:
            sbtn.disabled = False
            ctx.camera.release()
            ctx.camera_is_running = False
            ctx.camera = None
        
    
    camera_thread = threading.Thread(target=_video_capture, name='camera')
    camera_thread.daemon = True
    camera_thread.start()

    
def nbon_stop_camera(ctx, sbtn):
    ctx.logger('nbon_stop_camera()')
    if ctx.camera:
        ctx.camera_is_running = False
        for i in range(3):
            if ctx.camera is None:
                break
            time.sleep(1)
        if ctx.camera:
            ctx.camera.release()

In [ ]:
schema = {
    'type': 'page',
    'objs': [
        {
            'type': 'tab',
            'objs': [
                {
                    'name': 'Configuration',
                    'objs': [
                        {
                            'type': 'H',
                            'name': 'Background Subtraction',
                            'objs': [
                                nbeasy_widget_stringenum('cfg.bgsub_method', 'BG Sub Method', 1, enums=['None', 'MOG2', 'KNN'], width=300),
                                nbeasy_widget_int('cfg.bgsub_history', 'History Length', '300', min_=100, max_=600),
                                nbeasy_widget_bool('cfg.bgsub_det_shadow', 'Detect Shadows', True)
                            ]
                        },
                        {
                            'type': 'H',
                            'name': 'Remove Noise',
                            'objs': [
                                nbeasy_widget_int('cfg.rmn_shadow_threshold', 'Shadow Threshold', 250, min_=1, max_=255),
                                nbeasy_widget_stringenum('cfg.rmn_morpho_kernel', 'Morpho Kernel', 0, enums=[('3', 3), ('5', 5), ('7', 7), ('9', 9)], width=300),
                                nbeasy_widget_int('cfg.rmn_erode_iter', 'Erode Iters', 1),
                                nbeasy_widget_int('cfg.rmn_dilate_iter', 'Dilate Iters', 1),
                            ]
                        },
                        {
                            'type': 'H',
                            'name': 'Find Contours',
                            'objs': [
                                nbeasy_widget_float('cfg.cnt_area_threshold', 'Area Threshold', 2000),
                            ]
                        },
                        {
                            'type': 'V',
                            'name': 'Camera',
                            'objs': [
                                {
                                    'type': 'H',
                                    'objs': [
                                        nbeasy_widget_button('cfg.btn_start_camera', 'Start', width=200, style='success'),
                                        nbeasy_widget_button('cfg.btn_stop_camera', 'Stop', width=200, style='success')
                                    ],
                                    'justify_content': 'center'
                                },
                                nbeasy_widget_image('__cfg.camera_frame', 'Frame', '', height=480)
                            ],
                            'align_items': 'center'
                        }
                    ]
                }
            ]
        },
        {
            'type': 'onclick',
            'objs': [
                {
                    'handler': nbon_start_camera,
                    'params': {
                        'sources': ['cfg.btn_start_camera'],
                        'targets': [
                            'cfg.bgsub_method',
                            'cfg.bgsub_history',
                            'cfg.bgsub_det_shadow',
                            'cfg.rmn_shadow_threshold',
                            'cfg.rmn_morpho_kernel',
                            'cfg.rmn_erode_iter',
                            'cfg.rmn_dilate_iter',
                            'cfg.cnt_area_threshold',
                            '__cfg.camera_frame'
                        ]
                    }
                },
                {
                    'handler': nbon_stop_camera,
                    'params': {
                        'sources': ['cfg.btn_stop_camera'],
                        'targets': [
                        ]
                    }
                }
            ]
        }
    ]
}

if G:
    if hasattr(easy, 'camera') and easy.camera:
        easy.camera_is_running = False
        time.sleep(1)
        if easy.camera:
            easy.camera.release()
easy = nbeasy_schema_parse(schema, debug=True)
G = easy

In [ ]:
raise

## UnDef-2

In [ ]:
from IPython.display import display, Image

In [ ]:
parameters = cv2.aruco.DetectorParameters_create()
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_5X5_50)

In [ ]:
display_handle = display(None, display_id=True)

try:
    cap = cv2.VideoCapture(0)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(width, height)
    ret, prev_frame= cap.read()
    while True:
        _, frame_bgr = cap.read()
        
        # frame_diff = cv2.absdiff(frame_bgr, prev_frame)
        # prev_frame = frame_bgr.copy()
        # gray = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
        # blur = cv2.GaussianBlur(gray, (5, 5), 0)   
        # thresh = cv2.threshold(blur, 10, 255, cv2.THRESH_BINARY)[1]
        # dilate = cv2.dilate(thresh, None, iterations=5)
        # contours = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

        # cv2.drawContours(frame_bgr, contours, -1, (0, 0, 0), 3)

        motion = 0
        # for cnt in contours:
        #     mask = np.zeros([height, width], dtype=np.uint8)
        #     area = cv2.contourArea(cnt)
        #     if area > 10000:
        #         motion += 1
        #         # rect = cv2.minAreaRect(cnt)
        #         # (x, y), (w, h), angle = rect
        #         # object_width = w / pixel_cm_ratio
        #         # object_height = h / pixel_cm_ratio
        #         x, y, w, h = cv2.boundingRect(cnt)
        #         x1 = x if x < 5 else x - 5
        #         y1 = y if y < 5 else y - 5
        #         x2 = x + w if (x + w + 5) > width else x + w + 5
        #         y2 = y + h if (y + h + 5) > height else y + h + 5
        #         # cv2.rectangle(frame_bgr, (x1, y1), (x2, y2), (255, 255, 255), -1)
        #         mask[y1:y2, x1:x2] = 255
        #         frame_mask = cv2.add(frame_bgr, np.zeros_like(frame_bgr, dtype=np.uint8), mask=mask)
                  
        if motion > -1:
            corners, ids, _ = cv2.aruco.detectMarkers(frame_bgr, aruco_dict, parameters=parameters)
            # cv2.aruco.drawDetectedMarkers(frame_bgr, corners, ids)
            if corners and len(corners) > 0: 
                aruco_perimeter = cv2.arcLength(corners[0], True)
                pixel_cm_ratio = aruco_perimeter / 20
                frame_gray = cv2.cvtColor(frame_bgr.copy(), cv2.COLOR_BGR2GRAY)
                img_blur = cv2.GaussianBlur(frame_gray, (5, 5), 0)
                img_edged = cv2.Canny(img_blur, 50, 100)
                img_edged = cv2.dilate(img_edged, None, iterations=1)
                img_edged = cv2.erode(img_edged, None, iterations=1)
                # frame_thresh = cv2.adaptiveThreshold(frame_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 19, 5)
                contours = cv2.findContours(img_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    # if area > 5000 and area < 153600:
                    if area > 1000 and area < 20000:
                        # cv2.drawContours(frame_bgr, [cnt], 0, color=(0, 0, 0), thickness=4)
                        rect = cv2.minAreaRect(cnt)
                        (x, y), (w, h), angle = rect
                        object_width = w / pixel_cm_ratio
                        object_height = h / pixel_cm_ratio
                        box = cv2.boxPoints(rect)
                        box = np.int0(box)
 
                        cv2.circle(frame_bgr, (int(x), int(y)), 10, (0, 0, 255), -1)
                        cv2.polylines(frame_bgr, [box], True, (100, 10, 0), 2)
                        # cv2.putText(frame_bgr, "{} cm".format(round(object_width, 2)), (int(x - 100), int(y - 20)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 2)
                        cv2.putText(frame_bgr, "{} cm".format(round(object_height, 2)), (int(x - 50), int(y + 15)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 2)
        
        _, frame = cv2.imencode('.jpeg', frame_bgr)
        display_handle.update(Image(data=frame.tobytes()))
except KeyboardInterrupt:
    pass
finally:
    cap.release()
    display_handle.update(None)

## References

- [旋转矩阵](https://slash-honeydew-c53.notion.site/a88e94293aeb4b54a729ceeb2f40a353)

1. Part 1: Image formation and pinhole model of the camera - https://towardsdatascience.com/image-formation-and-pinhole-model-of-the-camera-53872ee4ee92
2. Part 2: Camera Extrinsic Matrix in Python - https://towardsdatascience.com/camera-extrinsic-matrix-with-example-in-python-cfe80acab8dd
3. Part 3: Camera Intrinsic Matrix in Python - https://towardsdatascience.com/camera-intrinsic-matrix-with-example-in-python-d79bf2478c12
4. Part 4: Find the Minimum Stretching Direction of Positive Definite Matrices - https://towardsdatascience.com/find-the-minimum-stretching-direction-of-positive-definite-matrices-79c2a3b397fc
5. Part 5: Camera Calibration in Python - https://towardsdatascience.com/camera-calibration-with-example-in-python-5147e945cdeb
You can also find all the code in the GitHub repository - https://github.com/wingedrasengan927/Image-formation-and-camera-calibration


https://programtalk.com/vs4/python/OteRobotics/realant/pose_estimation.py/

https://its201.com/article/libo1004/110851205